In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:

from src.model_analysis import *
from src.utils import *

# import scipy as sp
# import scipy.stats as stats
# import Orange

import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
from itertools import permutations

2024-04-25 06:57:52.273008: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-25 06:57:52.505883: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-25 06:57:52.574276: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-25 06:57:52.574290: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [3]:
CLASSIFIERS = ["LR","DT","SVM","XGB","MLP","RF"]
FEATURES =  ['tf', 'tfidf',"fastText","bert","glove", 'roberta']
METRICS = ['accuracy', 'auc', 'f1_score','fned', 'fped', 'subgroup', 'bpsn', 'bnsp']
DATASETs = ["HE", "DV", "WH"]

In [4]:
dataset_name = "HE"

path = f"results/{dataset_name}/classifiers/metrics.csv"
path2 = f"results/{dataset_name}/classifiers/metrics_v2.csv"

df = pd.read_csv(path)
df2 = pd.read_csv(path2)

df2 = df2.drop(df2[df2['model']=='LR_falcon'].index)
df2 = df2.drop(df2[df2['model']=='DT_falcon'].index)

df3 = pd.concat([df, df2], ignore_index=True)

df3 = df3.drop(columns="Unnamed: 0")

df3.to_csv(path)

## Analysing results 

In [5]:
def result_analysis(pvalue, alpha = 0.05):  
  if pvalue > alpha:
    result = "There is not significant difference between the performance of the methods"
  else:
    result = "There is significant difference between the performance of the methods."
  return result

# Wilcoxon statistic

In [6]:
def test_pair_wilcoxon(df1, df2, alternative="less", zero_method="wilcox"):
  p_array = [df1,df2]
  stat, pvalue = stats.wilcoxon(*p_array, alternative=alternative, zero_method=zero_method)
  # print ("Wilcoxon Statistics: {:.3f} | P-value: {:.3}".format(stat, pvalue))
  alpha = 0.05
  if pvalue > alpha:
    result = "not"#f"The classifier is not significantly {alternative} than the other"
    # result = ""
  else:
    result = "yes"# f"The classifier is significantly {alternative} than the other"
  # print(result)
  # print('\n','-------'*10)
  return pvalue, result

In [7]:
def get_metric_data(df, metric, classifier):
     #  Split column of lists into two new columns
    df[["classifier", "feature"]] = df.model.str.split("_", expand = True)
    df = df[df["classifier"] == classifier]

    features = df.feature.unique()

    df_result = pd.DataFrame()

    for ft in features:
        df_ft = df[df["feature"] == ft]
        df_result[ft] = df_ft[metric].values
    return df_result

In [54]:
def get_results(dataset_name, df, alternative):
    result = {}

    perm = permutations(FEATURES,2)

    for ft_1, ft_2 in list(perm):
        scores_ft1 = df[ft_1]
        scores_ft2 = df[ft_2]

        if dataset_name == "DV":
            pvalue, output = test_pair_wilcoxon(scores_ft1, scores_ft2, alternative=alternative, zero_method="zsplit")
        else:
            pvalue, output = test_pair_wilcoxon(scores_ft1, scores_ft2, alternative=alternative, zero_method="zsplit")
        result[f"{ft_1}-{ft_2}"] = f"({output,pvalue})"
    return result 

In [63]:
METRICS = ["subgroup"]#["fned", "fped","subgroup", auc,"f1_score"] 
dataset_name = "WH"

path = f"results/{dataset_name}/classifiers/metrics.csv"
df = pd.read_csv(path)

for met in METRICS:
  
  df_stats = pd.DataFrame()
  ft_names, ft_values = [], []
  perm = permutations(FEATURES,2)
  print( "-"*2,met.upper(), "\n")

  #check metrics
  if met in ["fned", "fped"]:
     alternative = "less"
  else:
     alternative = "greater"

  for clf_name in CLASSIFIERS:
      #get mean for the clf
      df_metric_clf = get_metric_data(df, met, clf_name)

      result = get_results(dataset_name, df_metric_clf, alternative)
      
      #  result = get_results(dataset_name, filename, clf_name, alternative)
      #df_stats = df_stats.append(result, ignore_index=True)
      result = pd.DataFrame(result, index=[0] )
      
      df_stats = pd.concat([df_stats, result], ignore_index=True)
   
  df_stats["classifier"] = CLASSIFIERS   

  display(df_stats)

-- SUBGROUP 



,tf-tfidf,tf-fastText,tf-bert,tf-glove,tf-roberta,tfidf-tf,tfidf-fastText,tfidf-bert,tfidf-glove,tfidf-roberta,...,glove-tfidf,glove-fastText,glove-bert,glove-roberta,roberta-tf,roberta-tfidf,roberta-fastText,roberta-bert,roberta-glove,classifier
0,"(('not', 1.0))","(('not', 1.0))","(('not', 1.0))","(('not', 0.6875))","(('not', 1.0))","(('yes', 0.03125))","(('not', 0.96875))","(('not', 1.0))","(('yes', 0.03125))","(('not', 0.90625))",...,"(('not', 1.0))","(('not', 1.0))","(('not', 1.0))","(('not', 0.96875))","(('yes', 0.03125))","(('not', 0.15625))","(('not', 1.0))","(('not', 0.90625))","(('not', 0.0625))",LR
1,"(('not', 0.40625))","(('not', 0.3125))","(('not', 0.59375))","(('not', 0.40625))","(('not', 0.3125))","(('not', 0.6875))","(('not', 0.59375))","(('not', 0.6875))","(('not', 0.5))","(('not', 0.3125))",...,"(('not', 0.59375))","(('not', 0.59375))","(('not', 0.90625))","(('not', 0.6875))","(('not', 0.78125))","(('not', 0.78125))","(('not', 0.6875))","(('not', 0.9375))","(('not', 0.40625))",DT
2,"(('not', 1.0))","(('not', 1.0))","(('not', 1.0))","(('not', 0.59375))","(('not', 0.96875))","(('yes', 0.03125))","(('not', 0.96875))","(('not', 1.0))","(('yes', 0.03125))","(('not', 0.5))",...,"(('not', 1.0))","(('not', 1.0))","(('not', 1.0))","(('not', 1.0))","(('not', 0.0625))","(('not', 0.59375))","(('not', 0.90625))","(('not', 1.0))","(('yes', 0.03125))",SVM
3,"(('not', 0.5))","(('not', 0.90625))","(('not', 1.0))","(('not', 1.0))","(('not', 0.5))","(('not', 0.5))","(('not', 0.90625))","(('not', 1.0))","(('not', 1.0))","(('not', 0.5))",...,"(('yes', 0.03125))","(('yes', 0.03125))","(('not', 0.0625))","(('yes', 0.03125))","(('not', 0.59375))","(('not', 0.59375))","(('not', 0.90625))","(('not', 1.0))","(('not', 1.0))",XGB
4,"(('not', 1.0))","(('not', 1.0))","(('not', 1.0))","(('not', 0.09375))","(('not', 0.96875))","(('yes', 0.03125))","(('not', 1.0))","(('not', 1.0))","(('not', 0.0625))","(('not', 0.84375))",...,"(('not', 0.96875))","(('not', 1.0))","(('not', 1.0))","(('not', 1.0))","(('not', 0.0625))","(('not', 0.21875))","(('not', 0.96875))","(('not', 0.90625))","(('yes', 0.03125))",MLP
5,"(('not', 0.90625))","(('not', 0.96875))","(('not', 0.96875))","(('not', 1.0))","(('not', 0.96875))","(('not', 0.15625))","(('not', 0.90625))","(('not', 1.0))","(('not', 0.90625))","(('not', 0.40625))",...,"(('not', 0.15625))","(('not', 0.59375))","(('not', 0.96875))","(('not', 0.0625))","(('not', 0.0625))","(('not', 0.6875))","(('not', 1.0))","(('not', 1.0))","(('not', 0.96875))",RF


In [64]:
print(dataset_name)

for clf in CLASSIFIERS:     

    df = df_stats[df_stats['classifier']==clf]

    for ft in FEATURES:
        
        cols = [c for c in df.columns if c.startswith(f"{ft}-")]
       

        values = df[cols].values[0]

        #print(values)
        if  all('not' not in v for v in values):     
            
            print("\n CLASSIFIER", clf)
            print("FEATURE: ", ft) 
            print(values) 

WH

 CLASSIFIER SVM
FEATURE:  bert
["(('yes', 0.03125))" "(('yes', 0.03125))" "(('yes', 0.03125))"
 "(('yes', 0.03125))" "(('yes', 0.03125))"]
